# 모델 & 토크나이저 로딩 기본기 및 실습 가이드

- 본 자료는 구글 코랩(Colab) 환경에서 Hugging Face `transformers` 라이브러리를 활용해 사전학습(Pre-trained) 모델을 로드하고, 토크나이저를 사용한 토큰화, 모델 Forward Pass, 임베딩 추출, 임베딩 유사도 계산 등을 실습해보는 예제를 포함하고 있습니다.

## 실습 목표
1. **Pre-trained Model 로드**: Hugging Face의 `from_pretrained()` 메서드를 통해 BERT 또는 DistilBERT 모델과 토크나이저를 로드하는 방법을 배운다.
2. **토크나이저 로딩 및 토큰화 기본 실습**: 한글 문장을 입력으로 하여 토큰화 과정을 살펴보고, token IDs를 확인한다.
3. **Forward Pass 실습**: 토큰화된 입력을 모델에 넣어서 출력(Logits, Hidden States)을 받아보고, 출력 구조를 이해한다.
4. **다양한 모델 로드 비교**: BERT, DistilBERT, GPT-2 등 다양한 모델을 로드해보고, 각각의 출력 형태 차이를 살펴본다.
5. **문장 임베딩 추출 실습**: BERT 기반 모델의 [CLS] 토큰 임베딩을 sentence-level 임베딩으로 활용하는 방법을 알아보고, 문장 간 유사도를 코사인 유사도로 계산하는 간단한 실습을 진행한다.
6. **경량 모델 비교**: DistilBERT 등의 경량 모델을 사용했을 때의 차이점도 살펴본다.



## 실습 준비

In [ ]:
!pip install transformers
!pip install sentencepiece  # 일부 토크나이저를 위해 필요할 수 있음

### 1. Pre-trained Model 로딩

**BERT 모델 & 토크나이저 로딩**

- transformers 라이브러리의 from_pretrained() 메서드를 사용하면 매우 간단히 사전학습 모델과 토크나이저를 로드할 수 있다.
- 여기서는 "bert-base-multilingual-cased" 모델을 예로 든다. 이 모델은 한국어를 비롯한 다양한 언어를 지원한다.

In [ ]:
from transformers import AutoTokenizer, AutoModel
import torch

# BERT 다국어 모델 로드(https://huggingface.co/google-bert/bert-base-multilingual-cased)
model_name = "bert-base-multilingual-cased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]

### 2. 토크나이저 로딩 및 토큰화 기본 실습

한글 문장을 토큰화해보고, input_ids를 확인

In [ ]:
sentence = "안녕하세요, 오늘 날씨가 참 좋네요."
encodings = tokenizer(sentence, return_tensors="pt")
print("Encodings:", encodings)
print("Input IDs:", encodings["input_ids"])
print("Attention Mask:", encodings["attention_mask"])
# token_type_ids는 BERT 구조상 문장쌍 입력 시 사용됨. 단일 문장일 경우 전부 0이거나 생략될 수 있음.

Encodings: {'input_ids': tensor([[   101,   9521, 118741,  35506,  24982,  48549,    117,   9580, 118762,
           8985,  49212,  11287,   9735,   9685,  77884,  48549,    119,    102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}
Input IDs: tensor([[   101,   9521, 118741,  35506,  24982,  48549,    117,   9580, 118762,
           8985,  49212,  11287,   9735,   9685,  77884,  48549,    119,    102]])
Attention Mask: tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])


위 출력 결과를 통해 토큰화된 결과(토큰 ID), attention mask 등을 확인할 수 있다.

토큰 아이디를 실제 토큰 문자열로 다시 매핑해보면 다음과 같이 할 수 있다

In [ ]:
tokens = tokenizer.convert_ids_to_tokens(encodings["input_ids"][0])
print("Tokens:", tokens)

Tokens: ['[CLS]', '안', '##녕', '##하', '##세', '##요', ',', '오', '##늘', '날', '##씨', '##가', '참', '좋', '##네', '##요', '.', '[SEP]']


### 3. 모델 Forward Pass 실습

In [ ]:
with torch.no_grad():
    outputs = model(**encodings)

# outputs은 Base BERT 모델일 경우, 일반적으로
# last_hidden_state와 pooler_output(일부 모델만), hidden_states, attentions 등을 포함할 수 있다.
# 여기서 outputs.last_hidden_state는 [batch_size, sequence_length, hidden_size] 형태의 텐서.
print("Last hidden state shape:", outputs.last_hidden_state.shape)

Last hidden state shape: torch.Size([1, 18, 768])


- last_hidden_state: 각 토큰별 임베딩(마지막 레이어 기준)
- pooler_output (BERT의 경우): [CLS] 토큰에 대한 임베딩을 추가로 변환한 벡터 (sentence-level representation을 제공하기 위해 사용)
- [CLS] 토큰은 단순한 시작 토큰을 넘어 문장 전체의 정보를 요약하도록 학습되는 경향이 있음.

In [ ]:
if hasattr(outputs, 'pooler_output'):
    print("Pooler output shape:", outputs.pooler_output.shape)
else:
    # 단, AutoModel로 로드한 Base BERT는 pooler_output을 포함하며,
    # AutoModelForMaskedLM 등의 다른 헤드 모델은 구조가 다를 수 있음.
    print("이 모델은 pooler_output을 제공하지 않습니다.")

Pooler output shape: torch.Size([1, 768])


### 4. 다양한 모델 로드해보기 (DistilBERT, GPT-2)

DistilBERT 로드
DistilBERT는 BERT보다 파라미터 수가 적고 빠른 추론이 가능하며, 성능은 비교적 비슷한 경량 모델이다.

In [ ]:
distil_name = "distilbert-base-multilingual-cased"
distil_tokenizer = AutoTokenizer.from_pretrained(distil_name)
distil_model = AutoModel.from_pretrained(distil_name)

distil_encodings = distil_tokenizer(sentence, return_tensors="pt")
with torch.no_grad():
    distil_outputs = distil_model(**distil_encodings)

print("DistilBERT last_hidden_state shape:", distil_outputs.last_hidden_state.shape)

# DistilBERT는 pooler_output이 없는 경우가 많다.
if hasattr(distil_outputs, 'pooler_output'):
    print("Pooler output shape:", distil_outputs.pooler_output.shape)
else:
    print("이 모델은 pooler_output을 제공하지 않습니다.")


DistilBERT last_hidden_state shape: torch.Size([1, 18, 768])
이 모델은 pooler_output을 제공하지 않습니다.


GPT-2 로드
- GPT-2는 Decoder 기반 언어 모델로, causal language modeling 목적으로 사전학습되었다.
- 한국어 지원 모델을 찾을 수도 있으나 여기서는 기본 gpt2 영문 모델을 예로 들어 출력 구조 차이를 살펴본다.

In [ ]:
gpt2_name = "gpt2"
gpt2_tokenizer = AutoTokenizer.from_pretrained(gpt2_name)
gpt2_model = AutoModel.from_pretrained(gpt2_name)

gpt2_encodings = gpt2_tokenizer("Hello world!", return_tensors="pt")
with torch.no_grad():
    gpt2_outputs = gpt2_model(**gpt2_encodings)

print("GPT-2 last_hidden_state shape:", gpt2_outputs.last_hidden_state.shape)
# GPT-2는 [batch_size, sequence_length, hidden_size] 형태의 last_hidden_state를 반환하며,
# pooler_output이나 CLS 토큰 개념이 없습니다.


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

GPT-2 last_hidden_state shape: torch.Size([1, 3, 768])


이처럼 모델별로 출력 형태와 활용 방식이 다를 수 있음을 확인할 수 있다.

### 5. 문장 임베딩 추출 실습

BERT 기반 모델에서 문장 임베딩을 얻는 한 가지 방법은 [CLS] 토큰 임베딩을 사용하는 것이다. [CLS] 토큰은 문장 전체를 대표하는 토큰으로 간주되며, BERT는 이를 통해 문장 단위 태스크를 수행하도록 학습되었다.

In [ ]:
with torch.no_grad():
    bert_outputs = model(**encodings)
# bert_outputs.last_hidden_state: [batch_size, seq_length, hidden_size]
# 첫 번째 토큰 [CLS] 임베딩을 추출
cls_embedding = bert_outputs.last_hidden_state[:, 0, :]  # [batch_size, hidden_size]
print("CLS embedding shape:", cls_embedding.shape)


CLS embedding shape: torch.Size([1, 768])


In [ ]:
bert_outputs.pooler_output[:,:10]

tensor([[ 0.1557,  0.0021,  0.2183,  0.0628,  0.1071,  0.4759,  0.0425,  0.2050,
         -0.3615,  0.3048]])

In [ ]:
outputs.pooler_output[:,:10]

tensor([[ 0.1557,  0.0021,  0.2183,  0.0628,  0.1071,  0.4759,  0.0425,  0.2050,
         -0.3615,  0.3048]])

cls_embedding는 해당 문장을 대표하는 임베딩으로 사용할 수 있다.

### 6. DistilBERT 등 경량 모델과의 비교

DistilBERT도 문장 임베딩용으로 사용 가능하나, pooler나 [CLS] 토큰 활용 방식이 다를 수 있으므로 보통 마지막 히든 스테이트의 평균(Mean Pooling) 등을 통해 문장 임베딩을 얻는다.

In [ ]:
with torch.no_grad():
    distil_outputs = distil_model(**distil_encodings)

# DistilBERT에는 CLS 토큰이 첫 토큰에 해당하나 pooler_output이 없으므로 마지막 히든 스테이트 첫 토큰을 사용하거나,
# 혹은 mean pooling을 사용할 수 있다.
distil_cls_embedding = distil_outputs.last_hidden_state[:, 0, :]
print("DistilBERT CLS embedding shape:", distil_cls_embedding.shape)


DistilBERT CLS embedding shape: torch.Size([1, 768])


### 7. 임베딩 간 유사도 계산 (코사인 유사도)

문장 임베딩을 얻었으니, 두 문장 간 유사도를 계산해보자. 코사인 유사도를 사용하면 쉽게 비교 가능하다.

In [ ]:
import torch
import torch.nn.functional as F

sentences = ["오늘 날씨가 좋네요.", "정말 맑고 화창한 날씨네요."]
encodings_1 = tokenizer(sentences[0], return_tensors="pt")
encodings_2 = tokenizer(sentences[1], return_tensors="pt")

with torch.no_grad():
    out1 = model(**encodings_1)
    out2 = model(**encodings_2)

cls_embed_1 = out1.last_hidden_state[:,0,:]
cls_embed_2 = out2.last_hidden_state[:,0,:]

# 코사인 유사도 계산
cos_sim = F.cosine_similarity(cls_embed_1, cls_embed_2)
print("Cosine similarity:", cos_sim.item())


Cosine similarity: 0.9361767768859863


- 코사인 유사도 값이 1에 가까울수록 매우 유사한 의미의 문장, -1에 가까울수록 매우 다른 의미의 문장임을 나타낸다.

- 유사한 문장 ("오늘 날씨가 좋네요." vs "정말 맑고 화창한 날씨네요.")의 경우 높은 유사도(0.8 이상)가 나올 것으로 기대할 수 있다.

### 8. 정리

- from_pretrained() 메서드를 통해 손쉽게 모델과 토크나이저를 로드할 수 있다.
- 토크나이저를 사용해 입력 문장을 토큰화하고, input_ids, attention_mask, token_type_ids 등을 확인할 수 있다.
- 다양한 모델(BERT, DistilBERT, GPT-2)을 로드하고 Forward Pass를 수행해보면, 각 모델의 출력 형식(히든 스테이트, pooler_output, etc.)에 대한 이해를 넓힐 수 있다.
- BERT 기반 모델에서 [CLS] 토큰 임베딩을 문장 임베딩으로 활용할 수 있으며, 임베딩 간 유사도(코사인 유사도) 계산을 통해 문장 간 의미적 유사도를 파악할 수 있다.
- DistilBERT 등의 경량 모델을 사용해도 유사한 과정을 수행할 수 있으며, 필요한 경우 mean pooling 등 다른 방법으로 문장 임베딩을 얻을 수 있다.







# BERT 모델 파인튜닝을 활용한 챗봇 만들기

## BERT란?
BERT(Bidirectional Encoder Representations from Transformers)는 구글에서 개발한 사전 훈련된 자연어 처리 모델로, 다양한 언어 이해 태스크에서 뛰어난 성능을 발휘합니다. BERT의 핵심은 트랜스포머 아키텍처의 양방향 인코더를 사용하는 것입니다.

## 핵심 기술
### 트랜스포머 모델
트랜스포머는 '어텐션 메커니즘'을 사용하여 입력 데이터의 각 요소 간의 관계를 학습합니다. BERT는 이 트랜스포머의 인코더 구조만을 사용합니다.

### 양방향 훈련
BERT의 또 다른 중요한 특징은 양방향으로 문맥을 이해합니다. 이는 기존 단방향 또는 양방향이 제한된 모델들과 비교하여 문맥 파악에 더욱 효과적입니다.

## 사전 훈련 태스크
BERT는 두 가지 주요 태스크를 통해 사전 훈련됩니다.

<img src="https://velog.velcdn.com/images/tm011899/post/72149edb-a1f0-46e0-aebe-4e89c0a490ae/image.PNG" width="600">

1. **Masked Language Model (MLM)**
   - 무작위로 선택된 토큰을 마스킹하고, 마스킹된 토큰을 예측하도록 합니다. 이 과정에서 모델은 양방향 문맥을 고려해야만 정확한 예측이 가능합니다.
2. **Next Sentence Prediction (NSP)**
   - 두 개의 문장이 주어졌을 때, 두 번째 문장이 첫 번째 문장의 다음 문장인지 관계 없는 문장인지 예측합니다. 이 태스크는 특히 문서 수준의 이해와 관계 추론에 도움을 줍니다.

## 활용 예
BERT는 자연어 이해 관련 다양한 태스크에 적용될 수 있습니다. 예를 들어, 감성 분석, 질문 응답 시스템, 문장 분류 등에 활용되어 우수한 결과를 도출합니다.

## 성과
BERT는 출시되자마자 여러 벤치마크에서 최고 성능을 기록하며, NLP 분야에서 새로운 기준을 설정했습니다.



In [ ]:
!pip install datasets

In [ ]:
# 데이터 처리 및 학습을 위한 라이브러리 임포트
from transformers import BertTokenizer, EncoderDecoderModel, EncoderDecoderConfig, Trainer, TrainingArguments,AutoConfig
from datasets import load_dataset, Dataset
import pandas as pd                 # 데이터프레임과 CSV 파일 처리를 위한 라이브러리
import urllib.request               # URL에서 파일 다운로드를 위한 라이브러리
import time                         # 학습 시간 측정을 위한 라이브러리
import numpy as np                  # 수학 계산 및 배열 처리를 위한 라이브러리
import matplotlib.pyplot as plt     # 그래프 및 데이터 시각화를 위한 라이브러리
import torch
import re                           # 정규 표현식 처리를 위한 라이브러리

# 챗봇 데이터 다운로드 및 로드
urllib.request.urlretrieve(
    "https://raw.githubusercontent.com/songys/Chatbot_data/master/ChatbotData.csv",
    filename="ChatBotData.csv")  # URL에서 챗봇 데이터를 로컬로 다운로드

# BERT 토크나이저와 모델을 초기화합니다.
tokenizer = BertTokenizer.from_pretrained('klue/bert-base')
config = EncoderDecoderConfig.from_encoder_decoder_configs(
    encoder_config=AutoConfig.from_pretrained('klue/bert-base'),
    decoder_config=AutoConfig.from_pretrained('klue/bert-base')
)
# 디코더의 시작 토큰 ID를 설정합니다.
config.decoder_start_token_id = tokenizer.cls_token_id  # CLS 토큰을 시작 토큰으로 사용
# 패딩 토큰 ID도 설정할 필요가 있습니다.
config.pad_token_id = tokenizer.pad_token_id

# 설정된 구성을 바탕으로 모델을 초기화합니다.
model = EncoderDecoderModel(config=config)

# GPU 사용이 가능한 환경이라면, 모델을 GPU로 이동시킵니다.
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# CSV 파일에서 데이터를 로드하고, 결측치를 확인 및 처리합니다.
df = pd.read_csv('ChatBotData.csv')
# 중복데이터 제거
df = df.drop_duplicates(subset='Q', keep='first')
print('챗봇 샘플의 개수 :', len(df))


# 질문과 답변 데이터 전처리
questions = df['Q'].apply(lambda x: re.sub(r"([?.!,])", r" \1 ", x).strip())
answers = df['A'].apply(lambda x: re.sub(r"([?.!,])", r" \1 ", x).strip())

# 토크나이징 및 데이터셋 변환
def tokenize_function(examples):
    model_inputs = tokenizer(examples['questions'], max_length=40, truncation=True, padding='max_length')
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples['answers'], max_length=40, truncation=True, padding='max_length')['input_ids']
    model_inputs['labels'] = labels
    return model_inputs

# 데이터셋을 훈련 및 검증용으로 나눕니다.
train_size = int(0.9 * len(questions))
train_dataset = Dataset.from_dict({'questions': questions[:train_size], 'answers': answers[:train_size]})
val_dataset = Dataset.from_dict({'questions': questions[train_size:], 'answers': answers[train_size:]})

train_dataset = train_dataset.map(tokenize_function, batched=True)
val_dataset = val_dataset.map(tokenize_function, batched=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/289 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/248k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/495k [00:00<?, ?B/s]

...

Config of the encoder: <class 'transformers.models.bert.modeling_bert.BertModel'> is overwritten by shared encoder config: BertConfig {
  "_name_or_path": "klue/bert-base",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.46.3",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 32000
}

Config of the decoder: <class 'transformers.models.bert.modeling_bert.BertLMHeadModel'> is overwritten by shared decoder config: BertConfig {
  "_name_or_path": "klue/bert-base",
  "add_cross_attention": true,
  "architectures": [
    "BertForMaskedLM"
  ],
  "attent

챗봇 샘플의 개수 : 11662


Map:   0%|          | 0/10495 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:4114: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/1167 [00:00<?, ? examples/s]

In [ ]:
# 트레이너 설정을 정의합니다.
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=20,
    per_device_train_batch_size=128,
    per_device_eval_batch_size=256,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    evaluation_strategy="epoch"  # 각 에폭마다 검증 데이터로 평가
)

# 트레이너 인스턴스를 생성하고 모델을 훈련합니다.
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset
)
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


/usr/local/lib/python3.10/dist-packages/transformers/models/encoder_decoder/modeling_encoder_decoder.py:629: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than tensor.new_tensor(sourceTensor).
  decoder_attention_mask = decoder_input_ids.new_tensor(decoder_input_ids != self.config.pad_token_id)
/usr/local/lib/python3.10/dist-packages/transformers/models/encoder_decoder/modeling_encoder_decoder.py:649: FutureWarning: Version v4.12.0 introduces a better way to train encoder-decoder models by computing the loss inside the encoder-decoder framework rather than in the decoder itself. You may observe training discrepancies if fine-tuning a model trained with versions anterior to 4.12.0. The decoder_input_ids are now created based on the labels, no need to pass them yourself anymore.
  warnings.warn(DEPRECATION_WARNING, FutureWarning)


Epoch,Training Loss,Validation Loss
1,9.542400,9.344317
2,8.137200,7.869624
3,6.226400,5.880588
4,3.873900,3.499101
5,1.345000,1.416778
6,0.886800,1.033488
7,0.803700,0.972997
8,0.714200,0.945585
9,0.656600,0.934352
10,0.623800,0.922413


/usr/local/lib/python3.10/dist-packages/transformers/models/encoder_decoder/modeling_encoder_decoder.py:629: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than tensor.new_tensor(sourceTensor).
  decoder_attention_mask = decoder_input_ids.new_tensor(decoder_input_ids != self.config.pad_token_id)
/usr/local/lib/python3.10/dist-packages/transformers/models/encoder_decoder/modeling_encoder_decoder.py:649: FutureWarning: Version v4.12.0 introduces a better way to train encoder-decoder models by computing the loss inside the encoder-decoder framework rather than in the decoder itself. You may observe training discrepancies if fine-tuning a model trained with versions anterior to 4.12.0. The decoder_input_ids are now created based on the labels, no need to pass them yourself anymore.
  warnings.warn(DEPRECATION_WARNING, FutureWarning)
/usr/local/lib/python3.10/dist-packages/tra

TrainOutput(global_step=1640, training_loss=2.0652259881903485, metrics={'train_runtime': 3930.0477, 'train_samples_per_second': 53.409, 'train_steps_per_second': 0.417, 'total_flos': 1.0059763181568e+16, 'train_loss': 2.0652259881903485, 'epoch': 20.0})

In [ ]:
# 사용자 입력에 대해 모델을 사용하여 응답을 생성하는 함수를 정의합니다.
def chat(question):
    inputs = tokenizer(question, return_tensors='pt', max_length=40, truncation=True, padding="max_length")
    inputs.to(device)
    model.eval()
    with torch.no_grad():
        outputs = model.generate(inputs['input_ids'])
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

In [ ]:
# 챗봇 테스트 예시
print(chat("고민이 있어"))

잘할 수 있을 거예요.


In [ ]:
print(chat("미래는 어떨까?"))

잘할 수 있을 거예요.


In [ ]:
print(chat("카페갈래?"))

좋은 곳으로 가보세요.


In [ ]:
print(chat("너무 화가나"))

자신을 더 사랑해주세요.
